# Load the AB-test data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

In [ ]:
df=pd.read_csv('/content/cookie_cats.csv')
df.head()

#Data cleaning :check missing data

In [ ]:
df.info()

#Count the number of players in each group

In [ ]:
df.groupby('version').count()#The number of players is roughly the same in the control gate_30 and test group gate_40.

#Analyzing Player Behavior

In [ ]:
df['sum_gamerounds'].plot.box(figsize=(5,10))

plot a boxplot to visualize the distribution of 'sum_gamerounds', which would give us a rough idea of how many game rounds were played by a player during the first week after installing the game.there is an outlier. This player played about 50,000 game rounds during the first week! most of the players only played about 5000games rounds during the first week.
Due to this outlier, this boxplot is extremely skewed and not very informative.

In [ ]:
df['sum_gamerounds'].describe()

In [ ]:
# Counting the number of players for each number of gamerounds
plot_df=df.groupby('sum_gamerounds').count().reset_index()
plot_df.head(20)

50% of players played fewer than 16 game rounds during the first week after installation, and 75% of players played fewer than 51 rounds.

3994 players did not even play a single round after installation.

Possible reasons may include:

-They downloaded a number of new games at the same time and were attracted by other games.

-They opened the app but did not like the design/interface/music, so they quit even before playing the game.

-They have not started playing the game yet.
......

Another number worth attention is that more than 14,000 (3994+5538+4606)players played fewer than three rounds. For these players, the reasons for leaving may include:

-They did not enjoy the game. (This is probably the most common reason).

-The game turned out to be different from what they expected.

-The game was too easy and they got bored of it.
......

It is important to understand why a large number of players quit the game at an early stage. Tactile Entertainment can try to collect player feedback, for example, through an in-app survey.

In [ ]:
# Counting the number of players for each number of gamerounds
plot_df = df.groupby('sum_gamerounds').count().reset_index()

# Plotting the distribution of players that played 0 to 100 game rounds
ax = plot_df.head(n=100).plot('sum_gamerounds', 'userid')
ax.set_xlabel("Gamerounds")
ax.set_ylabel("Count of Players")
ax.set_title("Game Rounds Played during First Week");

 The distribution is highly skewed, with a long tail on the right. A huge number of players played fewer than 20 rounds and left the game. For rounds greater than 60, the number of players stayed steady at about 300.

#Comparing 1-day Retention

A common metric in the video gaming industry for how fun and engaging a game is 1-day retention: the percentage of players that comes back and plays the game one day after they have installed it. The higher 1-day retention is, the easier it is to retain players and build a large player base.

In [ ]:
df['retention_1'].sum()

In [ ]:
df['retention_1'].count()

In [ ]:
df['retention_1'].sum() / df['retention_1'].count() # When using .sum(), T/F will first be converted to 1/0. true 1,false 0 不理解

# Equivalent to df['retention_1'].mean()
# Mean is calculated by summing the values and dividing by the total number of values.

 less than half of the players come back one day after installing the game. Now that we have a benchmark, let's look at how 1-day retention differs between the two AB-groups.

#1-day retention for each AB-group

In [ ]:
df.groupby('version')['retention_1'].mean()


Bootstrapping: Should we be confident in the difference?

In [ ]:
df.sample(frac = 1,replace = True)

In [ ]:
df.sample(frac = 1,replace = True).groupby('version')['retention_1'].mean()

In [ ]:
# Creating an list with bootstrapped means for each AB-group
boot_1d = []
for i in range(1000):
    boot_mean = df.sample(frac = 1,replace = True).groupby('version')['retention_1'].mean()
    boot_1d.append(boot_mean)

# Transforming the list to a DataFrame
boot_1d = pd.DataFrame(boot_1d)

# A Kernel Density Estimate plot of the bootstrap distributions
boot_1d.plot(kind='density');

These two distributions above represent the bootstrap uncertainty over what the underlying 1-day retention could be for the two AB-groups. There seems to be some evidence of a difference, albeit small. Let's plot the % difference to have a closer look.

In [ ]:
# Adding a column with the % difference between the two AB-groups
boot_1d['diff'] = (boot_1d.gate_30 - boot_1d.gate_40)/boot_1d.gate_40*100

In [ ]:
# Ploting the bootstrap % difference
ax = boot_1d['diff'].plot(kind='density')
ax.set_title('% difference in 1-day retention between the two AB-groups')

In [ ]:
# Calculating the probability that 1-day retention is greater when the gate is at level 30
print('Probability that 1-day retention is greater when the gate is at level 30:',(boot_1d['diff'] > 0).mean());

From this chart, we can see that the most likely % difference is around 1% - 2%, and that 96% of the distribution is above 0%, in favor of a gate at level 3

#Comparing 7-days Retention

The bootstrap analysis tells us that there is a high probability that 1-day retention is better when the gate is at level 30. However, since players have only been playing the game for one day, it is likely that most players haven't reached level 30 yet. That is, many players won't have been affected by the gate, even if it's as early as level 30.

But after having played for a week, more players should have reached level 40, and therefore it makes sense to also look at 7-day retention.

7-days Retention

In [ ]:
df['retention_7'].sum()

In [ ]:
df['retention_7'].count()

In [ ]:
df['retention_7'].sum() / df['retention_7'].count() # When using .sum(), T/F will first be converted to 1/0. true 1,false 0 不理解

# Equivalent to df['retention_1'].mean()
# Mean is calculated by summing the values and dividing by the total number of values.

1-day Retention V.S 7-days Retention

In [ ]:
df.groupby('version')['retention_1'].sum() / df.groupby('version')['retention_1'].count()

In [ ]:
df.groupby('version')['retention_7'].sum() / df.groupby('version')['retention_7'].count()

Insights:

Like with 1-day retention, 7-day retention is slightly lower when the gate is at level 40 (18.2%) than when the gate is at level 30 (19.0%).
This difference is also larger than for 1-day retention, presumably because more players have had time to hit the first gate.
The overall 7-day retention is lower than the overall 1-day retention; fewer people play a game a week after installing than a day after installing.
But as before, let's use bootstrap analysis to figure out how certain we should be of the difference between the AB-groups.

Bootstrap analysis

In [ ]:
# Creating a list with bootstrapped means for each AB-group
boot_7d = []
for i in range(500):
    boot_mean = df.sample(frac=1,replace=True).groupby('version')['retention_7'].mean()
    boot_7d.append(boot_mean)

# Transforming the list to a DataFrame
boot_7d = pd.DataFrame(boot_7d)

# Adding a column with the % difference between the two AB-groups
boot_7d['diff'] = (boot_7d.gate_30 - boot_7d.gate_40)/boot_7d.gate_40*100

# Ploting the bootstrap % difference
ax = boot_7d['diff'].plot(kind='density')
ax.set_title('% difference in 7-day retention between the two AB-groups')

# Calculating the probability that 7-day retention is greater when the gate is at level 30
print('Probability that 7-day retention is greater when the gate is at level 30:',(boot_7d['diff'] > 0).mean());

#Conclusion

The bootstrap result tells us that there is strong evidence that 7-day retention is higher when the gate is at level 30 than when it is at level 40. The conclusion is: If we want to keep retention high — both 1-day and 7-day retention — we should not move the gate from level 30 to level 40.

There are, of course, other metrics we could look at, like the number of game rounds played or how much in-game purchases are made by the two AB-groups. But retention is one of the most important metrics. If we don't retain our player base, it doesn't matter how much money they spend in-game.

So, why is retention higher when the gate is positioned earlier? One could expect the opposite: The later the obstacle, the longer people are going to engage with the game. But this is not what the data tells us. The theory of hedonic adaptation can give one explanation for this.

In short, hedonic adaptation is the tendency for people to get less and less enjoyment out of a fun activity over time if that activity is undertaken continuously. By forcing players to take a break when they reach a gate, their enjoyment of the game is prolonged. But when the gate is moved to level 40, fewer players make it far enough, and they are more likely to quit the game because they simply got bored of it.

#EDA

In [ ]:
df.groupby('version',)['retention_1','retention_7'].mean()

In [ ]:
# Find the average number of game rounds for each version, and the standard deviation and the number of values

version_group = df.groupby("version").agg(["mean", "std", "count"])["sum_gamerounds"]
plt.figure(figsize=(10, 6))
plt.bar(df["version"].unique(), version_group["mean"], color = "grey")
plt.title("Average Game Rounds")
plt.ylabel("Average")
plt.xlabel("Version")
plt.show()

In [ ]:
# Create a bar graph for each version's seven-day retention rate

version_group_retention7 = df.groupby("version").agg(["count", "sum"])["retention_7"]
version_group_retention7["proportion"] = version_group_retention7["sum"]/version_group_retention7["count"]

plt.figure(figsize=(10, 6))
plt.bar(df["version"].unique(), version_group_retention7["proportion"], color = "brown")
plt.title("Seven Day Retention Rates")
plt.ylabel("Retention Rate")
plt.xlabel("Version")
plt.show()